In [ ]:
#@title 0. 로컬(colab) 한글 환경 설정
# 그래프에 한글을 표기하기 위한 설정 (폰트 설치 + ...)
!sudo apt-get update -qq
!sudo apt-get install -y fonts-nanum fonts-nanum-coding
!sudo fc-cache -fv
!rm -rf ~/.cache/matplotlib
!rm -rf /tmp/.cache/matplotlib

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  fonts-nanum fonts-nanum-coding
0 upgraded, 2 newly installed, 0 to remove and 57 not upgraded.
Need to get 10.3 MB of archives.
After this operation, 34.1 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 fonts-nanum all 20200506-1 [10.3 MB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 fonts-nanum-coding all 2.5-3 [4,988 B]
Fetched 10.3 MB in 2s (4,659 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 2.)
debconf: falling back to

In [ ]:
import getpass
svcKey = getpass.getpass("svcKey 입력: ")

svcKey 입력: ··········


In [ ]:
#@title 1. API 데이터 수집 테스트
import requests

# url = 'https://apis.data.go.kr/B552584/EvCharger/getChargerInfo?serviceKey=여러분의_키&pageNo=1&numOfRows=10&zcode=26'
url = 'http://apis.data.go.kr/B552584/EvCharger/getChargerInfo'
# url = 'http://apis.data.go.kr/B552584/EvCharger/getChargerStatus'

# svcKey는 decoding key (encoding 하지 않은 key라는 뜻에서 decoding key. 실제로 원래 키 -> 인코딩 -> 디코딩 은 아님); 환경설정에 기술
# svcKey = "tB...=="
params = {'serviceKey' : svcKey, 'pageNo' : '1', 'numOfRows' : '10', 'zcode' : '26' }  # zcode; 26=부산, 11=서울

response = requests.get(url, params=params)
print(response.content)

b'<?xml version="1.0" encoding="UTF-8" standalone="yes"?><response><header><resultCode>00</resultCode><resultMsg>NORMAL SERVICE.</resultMsg><totalCount>28337</totalCount><pageNo>1</pageNo><numOfRows>10</numOfRows></header><body><items><item><statNm>\xeb\xb6\x80\xec\x82\xb0 \xec\x96\xb4\xeb\xa6\xb0\xec\x9d\xb4 \xeb\x8c\x80\xea\xb3\xb5\xec\x9b\x90</statNm><statId>ME181207</statId><chgerId>01</chgerId><chgerType>06</chgerType><addr>\xeb\xb6\x80\xec\x82\xb0\xea\xb4\x91\xec\x97\xad\xec\x8b\x9c \xeb\xb6\x80\xec\x82\xb0\xec\xa7\x84\xea\xb5\xac \xec\x84\xb1\xec\xa7\x80\xea\xb3\xa1\xeb\xa1\x9c 3</addr><addrDetail>null</addrDetail><location>null</location><lat>35.1831134</lat><lng>129.0459801</lng><useTime>24\xec\x8b\x9c\xea\xb0\x84 \xec\x9d\xb4\xec\x9a\xa9\xea\xb0\x80\xeb\x8a\xa5</useTime><busiId>ME</busiId><bnm>\xed\x99\x98\xea\xb2\xbd\xeb\xb6\x80</bnm><busiNm>\xed\x99\x98\xea\xb2\xbd\xeb\xb6\x80</busiNm><busiCall>1661-9408</busiCall><stat>2</stat><statUpdDt>20251201201257</statUpdDt><lastTsdt

In [ ]:
#@title 2. 데이터 가져오기 위한 준비_1
# zcode에 대해, pageNo, numOfRows 를 지정해서 데이터를 가져오는 API 밖에 정의되어 있지 않네...
import xml.etree.ElementTree as ET

# Update params for the initial request
params['pageNo'] = '1'
params['numOfRows'] = '9999'
params['zcode'] = '26'

# Make the API request
response = requests.get(url, params=params)

# Parse the XML response
root = ET.fromstring(response.content)

# Extract totalCount
total_count = int(root.find('header').find('totalCount').text)

# Extract all item elements
all_items = root.find('body').find('items').findall('item')

print(f"Total count for zcode=26: {total_count}")
print(f"Number of items in the first page (up to 9999): {len(all_items)}")

Total count for zcode=26: 28337
Number of items in the first page (up to 9999): 9999


In [ ]:
import pandas as pd
import math
import requests
import xml.etree.ElementTree as ET

url = 'http://apis.data.go.kr/B552584/EvCharger/getChargerInfo'
data = []

numOfRows = 9999
pageNo = 1

while True:
    params = {
        'serviceKey': svcKey,
        'pageNo': str(pageNo),
        'numOfRows': str(numOfRows),
        'zcode': '26'
    }

    response = requests.get(url, params=params)
    root = ET.fromstring(response.content)

    items = root.find('body').find('items').findall('item')

    # 데이터가 없으면 종료
    if not items:
        break

    for item in items:
        row = {child.tag: child.text for child in item}
        data.append(row)

    print(f"{pageNo} 페이지 수집 완료 ({len(items)}개)")
    pageNo += 1

df = pd.DataFrame(data)
print(df.shape)
df.head()

1 페이지 수집 완료 (9999개)
2 페이지 수집 완료 (9999개)
3 페이지 수집 완료 (8339개)
(28337, 37)


,statNm,statId,chgerId,chgerType,addr,addrDetail,location,lat,lng,useTime,...,note,limitYn,limitDetail,delYn,delDetail,trafficYn,year,floorNum,floorType,maker
0,부산 어린이 대공원,ME181207,01,06,부산광역시 부산진구 성지곡로 3,null,null,35.1831134,129.0459801,24시간 이용가능,...,None,N,None,N,None,N,2018,1,F,시그넷
1,부산시 교육청,ME181208,01,06,부산광역시 부산진구 화지로 12,null,null,35.1761301,129.0639265,24시간 이용가능,...,None,N,None,N,None,N,2018,1,F,시그넷
2,부산아시아드보조경기장VIP주차장,ME181210,01,06,부산광역시 연제구 거제동 1312,null,null,35.1883518,129.0555415,05:00~24:00,...,None,N,None,N,None,N,2018,1,F,시그넷
3,부산아시아드보조경기장VIP주차장,ME181210,02,06,부산광역시 연제구 거제동 1312,null,null,35.1883518,129.0555415,05:00~24:00,...,None,N,None,N,None,N,2018,1,F,시그넷
4,부산아시아드보조경기장VIP주차장,ME181210,11,04,부산광역시 연제구 거제동 1312,null,null,35.1883518,129.0555415,05:00~24:00,...,None,N,None,N,None,N,2018,1,F,시그넷


In [ ]:
#@title 3. 데이터 가져오기 위한 준비_2
# numOfRows를 최대치로 했는데 전부 못 가져왔으니, 나누어서 데이터를 가져와야겠네. 몇번에 나누어서 가져오면 좋을까?
import math

num_of_rows = int(params['numOfRows'])
total_pages = math.ceil(total_count / num_of_rows)

print(f"Total pages to retrieve all data for zcode=26: {total_pages}")

Total pages to retrieve all data for zcode=26: 3


In [ ]:
#@title 5. 데이터 수집 확인
import pandas as pd

# Prepare a list to hold dictionaries of item data
data = []

# Iterate through each item element and extract relevant information
for item in all_items:
    item_data = {}
    for child in item:
        item_data[child.tag] = child.text
    data.append(item_data)

# Create a pandas DataFrame from the list of dictionaries
df = pd.DataFrame(data)
df[:5]

,statNm,statId,chgerId,chgerType,addr,addrDetail,location,lat,lng,useTime,...,note,limitYn,limitDetail,delYn,delDetail,trafficYn,year,floorNum,floorType,maker
0,부산 어린이 대공원,ME181207,01,06,부산광역시 부산진구 성지곡로 3,null,null,35.1831134,129.0459801,24시간 이용가능,...,None,N,None,N,None,N,2018,1,F,시그넷
1,부산시 교육청,ME181208,01,06,부산광역시 부산진구 화지로 12,null,null,35.1761301,129.0639265,24시간 이용가능,...,None,N,None,N,None,N,2018,1,F,시그넷
2,부산아시아드보조경기장VIP주차장,ME181210,01,06,부산광역시 연제구 거제동 1312,null,null,35.1883518,129.0555415,05:00~24:00,...,None,N,None,N,None,N,2018,1,F,시그넷
3,부산아시아드보조경기장VIP주차장,ME181210,02,06,부산광역시 연제구 거제동 1312,null,null,35.1883518,129.0555415,05:00~24:00,...,None,N,None,N,None,N,2018,1,F,시그넷
4,부산아시아드보조경기장VIP주차장,ME181210,11,04,부산광역시 연제구 거제동 1312,null,null,35.1883518,129.0555415,05:00~24:00,...,None,N,None,N,None,N,2018,1,F,시그넷


In [ ]:
#@title 1. 데이터 기본 정보 확인
# 데이터 샘플 확인
# print(df.head())
# print(df.tail())

# 전체 컬럼과 타입, 결측치 확인
# print(df.info())

# 행/열 개수
# print(df.shape)
print(df.count())

# 각 컬럼의 데이터 타입
# print(df.dtypes)

# 데이터 구조 파악
# print(df.columns)
# print(df.index)
# print(df.values)

# # 특정 조건의 데이터 샘플 확인; dtype이 object인 컬럼의 값을 정렬해서 출력하고 갯수를 표시
# columns_object_dtype = df.columns[df.dtypes=='object']
# # print(columns_object_dtype)
# for col in columns_object_dtype:
#     # print (sorted(df[col].unique()))
#     # unique_values = sorted(df[col].unique())
#     unique_values = df[col].unique()
#     print(f'{col}: {len(unique_values)}개')
#     print(unique_values, '\n')

# colab의 cell 안에서 모든 컬럼을 다 표시하기 위한 설정
# pd.set_option('display.max_columns', None)

statNm         28337
statId         28337
chgerId        28337
chgerType      28337
addr           28337
addrDetail     28337
location       28337
lat            28337
lng            28337
useTime        28133
busiId         28337
bnm            28337
busiNm         28337
busiCall       28337
stat           28337
statUpdDt      28205
lastTsdt       25631
lastTedt       25741
nowTsdt         5315
powerType          0
output         28075
method         27378
zcode          28337
zscode         28337
kind           28317
kindDetail     28317
parkingFree    28336
note            4516
limitYn        28337
limitDetail    21868
delYn          28337
delDetail         13
trafficYn      28337
year           28337
floorNum       28143
floorType      28143
maker          13593
dtype: int64


In [ ]:
print(df.shape)
print(len(df))
df.info()

(28337, 37)
28337
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28337 entries, 0 to 28336
Data columns (total 37 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   statNm       28337 non-null  object
 1   statId       28337 non-null  object
 2   chgerId      28337 non-null  object
 3   chgerType    28337 non-null  object
 4   addr         28337 non-null  object
 5   addrDetail   28337 non-null  object
 6   location     28337 non-null  object
 7   lat          28337 non-null  object
 8   lng          28337 non-null  object
 9   useTime      28133 non-null  object
 10  busiId       28337 non-null  object
 11  bnm          28337 non-null  object
 12  busiNm       28337 non-null  object
 13  busiCall     28337 non-null  object
 14  stat         28337 non-null  object
 15  statUpdDt    28205 non-null  object
 16  lastTsdt     25631 non-null  object
 17  lastTedt     25741 non-null  object
 18  nowTsdt      5315 non-null   object
 19  powerTy